In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_df=pd.read_csv("../input/intra-hall-competetion-2022/train_dataset.csv")
train_df

FileNotFoundError: [Errno 2] No such file or directory: '../input/intra-hall-competetion-2022/train_dataset.csv'

In [ ]:
test_df=pd.read_csv("../input/intra-hall-competetion-2022/test_dataset.csv")
test_df

,v.id,on road old,on road now,years,km,rating,condition,economy,top speed,hp,torque
0,801,691061,729248,3,140142,3,10,9,180,53,109
1,802,528256,773069,6,148374,2,7,13,137,65,90
2,803,589290,827825,5,58548,3,9,12,182,108,85
3,804,624433,875297,6,141733,1,5,15,155,74,113
4,805,667370,760930,6,69230,2,1,13,151,118,76
...,...,...,...,...,...,...,...,...,...,...,...
195,996,633238,743850,5,125092,1,6,11,171,95,97
196,997,599626,848195,4,83370,2,9,14,161,101,120
197,998,646344,842733,7,86722,1,8,9,196,113,89
198,999,535559,732439,2,140478,4,5,9,184,112,128


In [ ]:
sample=pd.read_csv("../input/intra-hall-competetion-2022/sample_submision.csv")
sample

,v.id,current price
0,801,0
1,802,0
2,803,0
3,804,0
4,805,0
...,...,...
195,996,0
196,997,0
197,998,0
198,999,0


In [ ]:
train_df.shape,test_df.shape

((1000, 12), (200, 11))

In [ ]:
train_df.dropna(inplace=True)
train_df.shape

(1000, 12)

In [ ]:
test_df.dropna(inplace=True)
test_df.shape

(200, 11)

In [ ]:
X=train_df.iloc[:,1:11].values
X

array([[535651, 798186,      3, ...,    177,     73,    123],
       [591911, 861056,      6, ...,    148,     74,     95],
       [686990, 770762,      2, ...,    181,     53,     97],
       ...,
       [646344, 842733,      7, ...,    196,    113,     89],
       [535559, 732439,      2, ...,    184,    112,    128],
       [590105, 779743,      5, ...,    199,     99,     96]])

In [ ]:
y=train_df.iloc[:,11:12].values




In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=42)
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)



In [ ]:
from sklearn.linear_model import LinearRegression
linreg=LinearRegression()
linreg.fit(X_train,y_train)
yprelin=linreg.predict(X_test)





In [ ]:
from sklearn.metrics import mean_squared_error
rmse1=np.sqrt(mean_squared_error(y_test,yprelin))
rmse1

8733.580958311357

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
polyreg=PolynomialFeatures(degree=3)
X_poly=polyreg.fit_transform(X_train)
polyreg.fit(X_poly,y_train)
linreg2=LinearRegression()
linreg2.fit(X_poly,y_train)
yprepoly=linreg2.predict(polyreg.fit_transform(X_test))
rmse2=np.sqrt(mean_squared_error(y_test,yprepoly))
rmse2

637.4493842651352

In [ ]:
from sklearn.tree import DecisionTreeRegressor
decreg=DecisionTreeRegressor(random_state=0)
decreg.fit(X_train,y_train)
ypredecreg=decreg.predict(X_test)
rmse3=np.sqrt(mean_squared_error(y_test,ypredecreg))
rmse3

28866.13553105957

In [ ]:
param ={
    "max_depth":[2,3,4,5,8,9],
    "min_samples_leaf":[1,3,5,2,4,6,7],"min_samples_split":[2,3,4,5,6,8,9],"max_leaf_nodes":[2,3,4,5]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(DecisionTreeRegressor(),param_grid=param,cv=15)

In [ ]:
grid.fit(X_train,y_train)

GridSearchCV(cv=15, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 3, 4, 5, 8, 9],
                         'max_leaf_nodes': [2, 3, 4, 5],
                         'min_samples_leaf': [1, 3, 5, 2, 4, 6, 7],
                         'min_samples_split': [2, 3, 4, 5, 6, 8, 9]})

In [ ]:
grid.best_params_

{'max_depth': 3,
 'max_leaf_nodes': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [ ]:
grid.best_score_

0.8346579191762313

In [ ]:
DTR=DecisionTreeRegressor(random_state=0,max_depth= 3, min_samples_leaf=1, min_samples_split= 2,max_leaf_nodes= 5)
DTR.fit(X_train,y_train)
y_pred=DTR.predict(X_test)
rmse4=np.sqrt(mean_squared_error(y_test,y_pred))
rmse4

50393.76895764481

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_estimators=10,random_state=0)
rfr.fit(X_train,y_train.ravel())
y_predict=rfr.predict(X_test)
rmse5=np.sqrt(mean_squared_error(y_test,y_predict))
rmse5



18387.575945574637

In [ ]:
param={
    "n_estimators":[50],"max_depth":[3], "min_samples_leaf":[4],"min_samples_split":[2],"max_leaf_nodes":[4]
    
}

In [ ]:
grid2=GridSearchCV(RandomForestRegressor(),param_grid=param,cv=15)
grid2.fit(X_train,y_train.ravel())
grid2.best_params_






{'max_depth': 3,
 'max_leaf_nodes': 4,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 50}

In [ ]:
RFR=RandomForestRegressor(n_estimators=50,random_state=0,max_depth= 3, min_samples_leaf=4, min_samples_split= 2,max_leaf_nodes= 4)
RFR.fit(X_train,y_train.ravel())
y_pred=RFR.predict(X_test)
rmse6=np.sqrt(mean_squared_error(y_test,y_pred))
rmse6




48670.47534172705

In [ ]:
reqx=test_df.iloc[:,1:11].values
#X1=sc.fit_transform(reqx)
#y1=linreg2.predict(polyreg.fit_transform(X1)) 1 20,000
#y1=decreg.predict(X1) 28,000
y1=linreg.predict(reqx) 
#y1=rfr.predict(X1)    23000
#y1=RFR.predict(X1)



In [ ]:

#q=np.array(y)
#p=np.array(sample['v.id'])
x1=sample.iloc[:,0].values
y1=y1.flatten()
submission=pd.DataFrame({'v.id':x1,'current price':y1})
submission.to_csv("submission.csv",index=None)
submission







,v.id,current price
0,801,2.484850e+10
1,802,2.038306e+10
2,803,3.417676e+10
3,804,2.690030e+10
4,805,3.330305e+10
...,...,...
195,996,2.532343e+10
196,997,3.217899e+10
197,998,3.300027e+10
198,999,2.034955e+10
